<center><font size='20' color='lightblue'>Examen Final</font></center>

<center><font size='5' color='grey'>Que sea lo que la suerte quiera</center>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from imblearn.over_sampling import RandomOverSampler

In [3]:
df = pd.read_csv(
    '../Data/cervix.csv',
    sep=';',
    na_values='?',
)

In [13]:
df

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,STDs,STDs:cervical condylomatosis,STDs:vaginal condylomatosis,STDs:syphilis,STDs:pelvic inflammatory disease,STDs:genital herpes,STDs:molluscum contagiosum,STDs:AIDS,STDs:HIV,STDs:Hepatitis B,STDs:HPV,Dx:Cancer
0,18,4.0,15.0,1.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,52,5.0,16.0,4.0,1.0,1.0,3.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,46,3.0,21.0,4.0,0.0,1.0,15.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,42,3.0,23.0,2.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,34,3.0,18.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
664,32,2.0,19.0,1.0,0.0,1.0,8.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
665,25,2.0,17.0,0.0,0.0,1.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
666,33,2.0,24.0,2.0,0.0,1.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


<font color='green' size='5'>1. Realizar feature selection</font>

In [5]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [9]:
corr_matrix = df.corr().abs()

In [10]:
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

/var/folders/bq/_bqydjv50jg7scm3v5rwy0280000gn/T/ipykernel_78835/1559577561.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))


In [11]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.6)]

In [12]:
df = df.drop(columns=to_drop)

<font color='green' size='5'>2. Dividir el dataset en train y test</font>

In [15]:
x = df.drop(columns=['Dx:Cancer'])
y = df['Dx:Cancer']

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

<font color='green' size='5'>3. Escalar el dataset</font>

In [35]:
scaler = StandardScaler()

In [36]:
x_scaled = scaler.fit_transform(x_train)

In [37]:
rf = RandomForestClassifier(random_state=32, criterion='entropy').fit(x_scaled, y_train)

In [38]:
scores = cross_val_score(rf, x_scaled, y_train, cv=4)

In [49]:
print("%0.2f accuracy with a standard deviation of %0.5f" % (scores.mean(), scores.std()))

0.97 accuracy with a standard deviation of 0.00367


In [40]:
predictions = rf.predict(scaler.transform(x_test))

In [41]:
confusion_matrix(y_test, predictions)

array([[197,   0],
       [  4,   0]])

In [42]:
recall_score(y_test, predictions)

0.0

In [90]:
rf_2 = RandomForestClassifier(random_state=32, criterion='gini').fit(x_scaled, y_train)

In [91]:
scores_2 = cross_val_score(rf_2, x_scaled, y_train, cv=4)

In [92]:
print("%0.2f accuracy with a standard deviation of %0.5f" % (scores_2.mean(), scores_2.std()))

0.97 accuracy with a standard deviation of 0.00010


In [93]:
predictions_2 = rf_2.predict(scaler.transform(x_test))

In [94]:
confusion_matrix(y_test, predictions_2)

array([[197,   0],
       [  4,   0]])

In [95]:
recall_score(y_test, predictions_2)

0.0

<font size='15' color='yellow'>Análisis de Resultados</font>

Tal como lo demuestran los resultados, ambos modelos presentan el mismo accuracy con una ligera diferencia en la varianza, por lo que no se puede concluir cual mejor modelo es mejor porque ambos producen los mismo resultados. Algo a destacar es que la matriz de confusión muestra que la diagonal no presenta aciertos en toda su extension, esto se puede deber a que el dataset no esta balanceado, hay muy pocos valores de 1 y muchos valores de 0, por lo que el modelo no puede aprender a predecir correctamente los valores de 1. Tambien se aprecia que el recall esta en 0, lo que significa que los datos de entrenamiento no son representativos del dataset.

<font size='5' color='green'>Posible solucion</font>

In [129]:
oversample = RandomOverSampler(sampling_strategy=0.5)

In [130]:
x_over, y_over = oversample.fit_resample(x, y)

In [131]:
x_over_train, x_over_test, y_over_train, y_over_test = train_test_split(x_over, y_over, test_size=0.3, random_state=32)

In [132]:
scaler_2 = StandardScaler()

In [133]:
x_over_scaled = scaler_2.fit_transform(x_over_train)

In [134]:
rf_3 = RandomForestClassifier(random_state=32, criterion='entropy').fit(x_over_scaled, y_over_train)

In [135]:
scores_3 = cross_val_score(rf_3, x_over_scaled, y_over_train, cv=4)

In [136]:
print("%0.2f accuracy with a standard deviation of %0.5f" % (scores_3.mean(), scores_3.std()))

0.99 accuracy with a standard deviation of 0.00959


In [137]:
predictions_3 = rf_3.predict(scaler_2.transform(x_over_test))

In [138]:
confusion_matrix(y_over_test, predictions_3)

array([[198,   1],
       [  0,  94]])

In [139]:
recall_score(y_over_test, predictions_3)

1.0

In [140]:
rf_4 = RandomForestClassifier(random_state=32, criterion='gini').fit(x_over_scaled, y_over_train)

In [141]:
scores_4 = cross_val_score(rf_4, x_over_scaled, y_over_train, cv=4)

In [142]:
print("%0.2f accuracy with a standard deviation of %0.5f" % (scores_4.mean(), scores_4.std()))

0.99 accuracy with a standard deviation of 0.00959


In [143]:
predictions_4 = rf_4.predict(scaler_2.transform(x_over_test))

In [144]:
confusion_matrix(y_over_test, predictions_4)

array([[198,   1],
       [  0,  94]])

In [145]:
recall_score(y_over_test, predictions_4)

1.0

<font size='15' color='yellow'>Análisis de Resultados</font>

Se aprecia el cambio, el accuracy aumenta en ambos modelos y la varianza se iguala por lo que no se puede concluir cual de los es mejor. Lo que si amerita mencion es el hecho de que con el oversampling de 0.5 el recall ya no esta en cero y ahora la matriz de confusion si presenta valores en toda su diagonal lo que significa que el modelo ya puede predecir correctamente los valores de 1. El inconveniente esta en que el accurracy y el recall de los modelos muestra que se pudo haber caido en un overfitting.